In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [11]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [15]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [16]:
def train_model(model, trainloader, criterion, optimizer, n_epoch):
    for epoch in range(n_epoch):  # loop over the dataset multiple times
    
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)
    
            # zero the parameter gradients
            optimizer.zero_grad()
    
            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0
    
    print('Finished Training')

In [18]:
def run_model(model, criterion, optimizer, batch_size, n_epoch=2):
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)
    train_model(model, trainloader, criterion, optimizer, n_epoch)
    test_model(model, testloader)

In [19]:
def test_model(model, testloader):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = model(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

в качестве функции потерь рассмотрим кросс-энтропию, в качестве метода оптимизации весов - стохастический градиентный спуск.
Будем менять гипер-параметры оптимизатора, размер батча, количество эпох.

In [23]:
import torch.optim as optim

batch_size = 4

net = Net()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
run_model(net, criterion, optimizer, batch_size)

[1,  2000] loss: 2.191
[1,  4000] loss: 1.859
[1,  6000] loss: 1.718
[1,  8000] loss: 1.647
[1, 10000] loss: 1.532
[1, 12000] loss: 1.471
[2,  2000] loss: 1.416
[2,  4000] loss: 1.358
[2,  6000] loss: 1.346
[2,  8000] loss: 1.308
[2, 10000] loss: 1.292
[2, 12000] loss: 1.264
Finished Training
Accuracy of the network on the 10000 test images: 53 %


In [24]:
import torch.optim as optim

batch_size = 8

net = Net()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
run_model(net, criterion, optimizer, batch_size, n_epoch=4)

[1,  2000] loss: 2.220
[1,  4000] loss: 1.788
[1,  6000] loss: 1.576
[2,  2000] loss: 1.456
[2,  4000] loss: 1.393
[2,  6000] loss: 1.346
[3,  2000] loss: 1.256
[3,  4000] loss: 1.257
[3,  6000] loss: 1.212
[4,  2000] loss: 1.160
[4,  4000] loss: 1.137
[4,  6000] loss: 1.134
Finished Training
Accuracy of the network on the 10000 test images: 60 %


In [25]:
import torch.optim as optim

batch_size = 8

net = Net()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())
run_model(net, criterion, optimizer, batch_size, n_epoch=4)

[1,  2000] loss: 1.776
[1,  4000] loss: 1.499
[1,  6000] loss: 1.418
[2,  2000] loss: 1.318
[2,  4000] loss: 1.294
[2,  6000] loss: 1.258
[3,  2000] loss: 1.189
[3,  4000] loss: 1.173
[3,  6000] loss: 1.172
[4,  2000] loss: 1.078
[4,  4000] loss: 1.124
[4,  6000] loss: 1.116
Finished Training
Accuracy of the network on the 10000 test images: 59 %


In [26]:
import torch.optim as optim

batch_size = 4

net = Net()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())
run_model(net, criterion, optimizer, batch_size, n_epoch=3)

[1,  2000] loss: 1.839
[1,  4000] loss: 1.563
[1,  6000] loss: 1.525
[1,  8000] loss: 1.438
[1, 10000] loss: 1.423
[1, 12000] loss: 1.400
[2,  2000] loss: 1.339
[2,  4000] loss: 1.312
[2,  6000] loss: 1.307
[2,  8000] loss: 1.303
[2, 10000] loss: 1.296
[2, 12000] loss: 1.279
[3,  2000] loss: 1.192
[3,  4000] loss: 1.209
[3,  6000] loss: 1.217
[3,  8000] loss: 1.216
[3, 10000] loss: 1.208
[3, 12000] loss: 1.212
Finished Training
Accuracy of the network on the 10000 test images: 55 %


Качество модели среднее, для улучшения модели можно применить более сложные свертки и пулинги или использовать предобученные модели. В рамках данной модели изменение количества эпох (увеличение количества), оптимизатора (подобрать более гибкий) в динамике показывают лучшие показатели обучения.